In [23]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import gurobipy
from gurobipy import quicksum

In [3]:
data = pd.read_csv('../dataset/UQ Tesla Battery-2020-5.csv')
data.columns

Index(['Measurement time', 'FCAS Event (%)', 'Full Charge Energy (kWh)',
       'Nominal Energy (kWh)', 'Expected Energy (kWh)', 'Charge P Max (kW)',
       'Discharge P Max (kW)', 'Available Blocks', '3 Phase Voltage (V)',
       '3 Phase Current (A)', '3 Phase Power (kW)',
       '3 Phase Reactive Power (kVAr)', '3 Phase Apparent Power (kVA)',
       'Power Factor', 'Frequency (Hz)', 'Real Energy Imported (kWh)',
       'Real Energy Exported (kWh)', 'Reactive Energy Imported (kVArh)',
       'Reactive Energy Exported (kVArh)', 'Apparent Energy (kVAh)',
       'Energy Price ($/MWh)', 'Raise 6sec Price ($/MWh)',
       'Raise 60sec Price ($/MWh)', 'Raise 5min Price ($/MWh)'],
      dtype='object')

In [225]:
df = data[data['Measurement time'].astype(str).str.contains('2020-04-04')]
columns = ['3 Phase Power (kW)', 'Measurement time', 'Energy Price ($/MWh)','Expected Energy (kWh)', 'FCAS Event (%)']
assert set(df['FCAS Event (%)'].values) == set(['0%'])
df = df[columns]
dr = np.average(df['3 Phase Power (kW)'].values.reshape(48, 6), axis=1)/1000
price = np.average(df['Energy Price ($/MWh)'].values.reshape(48, 6), axis=1)
remain = np.average(df['Expected Energy (kWh)'].values.reshape(48, 6), axis=1) / 2200
initial_e = df['Expected Energy (kWh)'].values[0]/1000/2.2
dr_p, dr_d, dr_e = generate_data(price, e0=initial_e, T=48, c1=20,c2=10, eta=0.86)
pred_dr = dr_p - dr_d

Set parameter NonConvex to value 2
Set parameter TimeLimit to value 600
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 337 rows, 193 columns and 2689 nonzeros
Model fingerprint: 0xd78c0a92
Model has 48 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [6e+00, 9e+01]
  QObjective range [2e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-02, 1e+00]
Presolve removed 289 rows and 53 columns
Presolve time: 0.01s
Presolved: 48 rows, 140 columns, 2396 nonzeros
Presolved model has 48 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.128e+03
 Factor NZ  : 1.176e+03
 Factor Ops : 3.802e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.836048

In [227]:
e = np.zeros(49)
charge = np.zeros(48)
discharge = np.zeros(48)
e[0] = df['Expected Energy (kWh)'].values[0]/1000/2.2
for i in range(48):
    charge[i] = max(0, dr[i])
    discharge[i] = -min(0, dr[i])
    e[i+1] = e[i] + charge[i] / 4.4 * 0.85 - discharge[i] / 4.4 / 0.95
fig = go.Figure()
fig.add_trace(go.Scatter(y=dr,name='True'))
fig.add_trace(go.Scatter(y=pred_dr,name='pred'))
fig.add_trace(go.Scatter(y=e,name='e'))
fig.add_trace(go.Scatter(y=dr_e,name='predicted_e'))
#fig.add_trace(go.Scatter(y=remain,name='remain'))
fig.add_trace(go.Scatter(y=price,name='price',yaxis='y2'))
fig.update_layout(
    yaxis1={'title':'Chargin Power'},
    yaxis2={'title':'Price ($/MWh)','anchor':'x', 'overlaying':'y', 'side':'right', 'showgrid':False},
)
fig.show()

In [21]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=dr,name='Charging'))
fig.add_trace(go.Scatter(y=price,yaxis='y2',name='price'))
fig.update_layout(
    yaxis1={'title':'Chargin Power'},
    yaxis2={'title':'Price ($/MWh)','anchor':'x', 'overlaying':'y', 'side':'right', 'showgrid':False},
)
fig.show()

In [213]:
def generate_data(price, e0, c1, c2, eta=0.86, T=48):
    P1 = 1.1
    P2 = 0
    E1 = 1
    E2 = 0
    e0 = e0

    # define the variable and objective function
    model = gurobipy.Model()
    p = model.addVars(T, name='p') # charging
    d = model.addVars(T, name='d') # discharging 
    e = model.addVars(T+1, name='e')
    c = model.addVars(T, name='c')
    
    for i in range(T+1):
        model.addConstr(e[i] == e0 + quicksum(p[j] / 4.4 * 0.85 - d[j] / 4.4 / 0.95 for j in range(i)))

    utility_cost = c2 * quicksum(d[i]*d[i] for i in range(T)) + c1*quicksum(p[i] for i in range(T))
    f = quicksum(price[i]*(p[i]-d[i]) for i in range(T)) + utility_cost
    # define constraints
    model.addConstrs(p[i] <= P1 for i in range(T))
    model.addConstrs(p[i] >= P2 for i in range(T))
    model.addConstrs(d[i] <= P1 for i in range(T))
    model.addConstrs(d[i] >= P2 for i in range(T))
    model.addConstrs(e[i] <= E1 for i in range(1,T+1))
    model.addConstrs(e[i] >= E2 for i in range(1,T+1))

    model.setObjective(f, gurobipy.GRB.MINIMIZE)
    #model.setParam('OutputFlag', 1)
    model.setParam("NonConvex", 2)
    model.setParam('TimeLimit', 600)
    model.optimize()

    dvalue = np.zeros(T)
    pvalue = np.zeros(T)
    evalue = np.zeros(T+1)
    for k, v in model.getAttr('x', p).items():
        pvalue[k] = v
    for k, v in model.getAttr('x', d).items():
        dvalue[k] = v
    for k, v in model.getAttr('x', e).items():
        evalue[k] = v
    
    return pvalue, dvalue, evalue

In [136]:
fig = go.Figure()
i = np.random.randint(40,50)
fig.add_trace(go.Scatter(y=train_dr[i],name='Charging'))
fig.add_trace(go.Scatter(y=train_price[i],yaxis='y2',name='price'))
#fig.add_trace(go.Scatter(y=data[288*i:288*(i+1)]['Frequency (Hz)'],yaxis='y2',name='frequency'))
fig.update_layout(
    yaxis1={'title':'Chargin Power','range':[-1.5,1.5]},
    yaxis2={'title':'Price ($/MWh)','anchor':'x', 'overlaying':'y', 'side':'right','range':[0,200], 'showgrid':False},
)
fig.show()

In [138]:
data = pd.read_csv('../dataset/UQ Tesla Battery - 2022 (5-min resolution).csv')
df_price = data['Energy Price ($/MWh)']
df_y = data['3 Phase Power (kW)']

T = 48 # one day
N_train = 40
N_test = 10
model = MLP(T,T)
train_price = np.mean(df_price[:N_train*T].values.reshape(N_train,T, -1), axis=2)
train_dr = np.mean(df_y[:N_train*T].values.reshape(N_train,T, -1), axis=2)/1000
test_price = np.mean(df_price[N_train*T:(N_test+N_train)*T].values.reshape(N_test,T, -1), axis=2)
test_dr = np.mean(df_y[N_train*T:(N_test+N_train)*T].values.reshape(N_test,T, -1), axis=2)/1000

history = model.fit(train_price,train_dr, validation_data = (test_price, test_dr),  epochs=1000, batch_size=32, verbose=0) #validation_data = (test_price, test_dr),
L = history.history['loss']
val_L = history.history['val_loss']

In [140]:
model2 = RNNmodel(T,T)
history2 = model2.fit(train_price,train_dr, validation_data = (test_price, test_dr),  epochs=1000, batch_size=32, verbose=0) #validation_data = (test_price, test_dr),
L2 = history.history['loss']
val_L2 = history.history['val_loss']

In [104]:
RNN_predict = model2.predict(test_price)
MLP_predict = model.predict(test_price)

In [139]:
val_L[-5:]

[0.2678595781326294,
 0.2678561806678772,
 0.2678542733192444,
 0.26785290241241455,
 0.2678532898426056]

In [119]:
fig = go.Figure()
i = np.random.randint(10)
fig.add_trace(go.Scatter(y=test_dr[i]*1000,name='True'))
#fig.add_trace(go.Scatter(y=test_price[i],yaxis='y2',name='price'))
fig.add_trace(go.Scatter(y=MLP_predict[i]*1000,name='MLP'))
fig.add_trace(go.Scatter(y=RNN_predict[i]*1000,name='RNN'))
#fig.add_trace(go.Scatter(y=data[288*i:288*(i+1)]['Frequency (Hz)'],yaxis='y2',name='frequency'))
fig.update_layout(
    yaxis1={'title':'Chargin Power','range':[-1200,1200]},
    yaxis2={'title':'Price ($/MWh)','anchor':'x', 'overlaying':'y', 'side':'right','range':[0,200], 'showgrid':False},
)
fig.show()

In [29]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
import tensorflow as tf
def RNNmodel(input_dim, output_dim):
    model = Sequential()
    model.add(LSTM(units=output_dim,return_sequences=True,input_shape=(input_dim, 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=output_dim,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=output_dim,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dropout(0.2))
    model.add(Dense(units=output_dim))
    model.compile(optimizer='adam',loss='mean_squared_error')
    return model

def MLP(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(output_dim))
    model.compile(optimizer='adam',loss='mean_squared_error')
    return model